In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import cv2

import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.utils import to_categorical

In [ ]:
DATADIR = r"E:\Images\Buildings"

CATEGORIES = ["EmpireStateBuilding", "SydneyOpera"]


training_data = []

def create_training_data():
    for category in CATEGORIES:  
        path = os.path.join(DATADIR, category)  
        class_num = CATEGORIES.index(category)  

        for img in os.listdir(path):            
            try:
                img_array = cv2.imread(os.path.join(path, img) ,cv2.IMREAD_GRAYSCALE)  
                new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))  
                training_data.append([new_array, class_num])             
            except Exception as e:                                       
                pass

            
create_training_data()

In [ ]:
X = []       # Feature-Set
y = []       # Label-Set


for features, label in training_data:
    X.append(features)
    y.append(label)
    
    
X = np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
X = X/255.0                                           # Normalize between 0 and 1.

y = np.array(y)

In [ ]:
# Build model: We use the CNN (Convolutional Neural Network) as a special neural net for image classification.

model = Sequential()

model.add(Conv2D(100, (3, 3), activation='relu', input_shape=X.shape[1:]))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(100, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())                          # converts 3D feature maps to 1D feature vectors

model.add(Dense(30, activation='relu'))       

model.add(Dense(1, activation='sigmoid'))


model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])


model.fit(X, y, batch_size=32, epochs=3)   

In [ ]:
# Classify a new picture

def prepare(filepath):
    IMG_SIZE = 50                                               
    img_array = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)      
    new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))    
    
    return new_array.reshape(-1, IMG_SIZE, IMG_SIZE, 1)        


# make prediction

prediction = model.predict([prepare(r"E:\Images\Test\esb.jpg")])        
print(prediction)                                       
print(prediction[0][0])                                

print(CATEGORIES[int(prediction[0][0])])     